傾向スコア（逆確率重み付け（IPTW））による因果推論

基本的な考え方としては，各人（特徴量X）の介入Zを取りうる確率で正規化してあげる，ということ

In [1]:
# 乱数シード
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)

In [2]:
# 使用するパッケージ（ライブラリと関数）を定義
# 標準正規分布の生成用
from numpy.random import *

# グラフの描画用
import matplotlib.pyplot as plt

# SciPy 平均0、分散1に正規化（標準化）関数
import scipy.stats

# シグモイド関数をimport
from scipy.special import expit

# その他
import pandas as pd

# 自前でデータ作成

In [4]:
num_data = 200

## 性別，年齢

In [5]:
#一様乱数
# age
x_1 = randint(15, 76, num_data) 
# 性別（0: 女性，1: 男性）
x_2 = randint(0, 2, num_data)

## テレビCMをみたか

In [6]:
# ノイズの生成
e_z = randn(num_data)

# シグモイド関数に入れる線形関数・・・（1）
z_base = x_1 + (1 - x_2)*10 - 40 + 5*e_z

# シグモイド関数の計算
z_prob = expit(0.1*z_base)

# テレビCMを見たかどうか（0: 見ていない，1: 見た）
Z = np.array([])

# （1）より年配の女性の方が見やすくなる
for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)

## 購入量Y

In [7]:
# ノイズの生成
e_y = randn(num_data)

# 若年層で，男性で，テレビを見ていると購入が多くなる
Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y 

# 作成データ

In [8]:
df = pd.DataFrame({
    'age': x_1,
    'sex': x_2,
    'cm': Z,
    'amount': Y
})

In [9]:
df.head()

,age,sex,cm,amount
0,62,0,1.0,24.464285
1,34,0,0.0,45.693411
2,53,1,1.0,64.998281
3,68,1,1.0,47.186898
4,27,1,0.0,100.114260


## CM別平均

In [10]:
print(df[df['cm']==1.0].mean())
print('------')
print(df[df['cm']==0.0].mean())

age       55.836066
sex        0.483607
cm         1.000000
amount    49.711478
dtype: float64
------
age       32.141026
sex        0.692308
cm         0.000000
amount    68.827143
dtype: float64


# 傾向スコアの推定

介入Zを目的変数として，ロジスティック回帰を用いて特徴量XのZの予測値を求める．

In [11]:
from sklearn.linear_model import LogisticRegression

# 説明変数
X = df[['age', 'sex']]

# 目的変数
Z = df['cm']

# 回帰
reg = LogisticRegression().fit(X, Z)

# 回帰実施後の係数
print('coef beta: ', reg.coef_)
print('coef alpah: ', reg.intercept_)

coef beta:  [[ 0.10562765 -1.38263933]]
coef alpah:  [-3.37146523]


## 各人の傾向スコアを求める

In [12]:
Z_pre = reg.predict_proba(X)
print(Z_pre[0:5]) # 5人ほどの結果を見てみる
print('----')
print(Z[0:5])

[[0.04002323 0.95997677]
 [0.44525168 0.55474832]
 [0.30065918 0.69934082]
 [0.08101946 0.91898054]
 [0.87013558 0.12986442]]
----
0    1.0
1    0.0
2    1.0
3    1.0
4    0.0
Name: cm, dtype: float64


Zは２クラスなので，２列目が大きいとcmを見ている傾向であると予測される．

## 平均処置効果ATEを求める

In [13]:
ATE_i = Y/Z_pre[:, 1]*Z - Y/Z_pre[:, 0]*(1-Z)
ATE = 1/len(Y)*ATE_i.sum()
print('推定されるATE', ATE)

推定されるATE 8.847476810855458
